In [2]:
import boto3
import pprint
from botocore.client import Config

In [4]:
pp = pprint.PrettyPrinter(indent=2)

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config)

In [5]:
plan_kb_id = "YOURKBID"

In [6]:
def create_filter(filter_type):
    filter_mappings = {
        "hmo_user": "hmo_plan.pdf", 
        "ppo_user": "ppo_plan.pdf", 
        "ppo_hd_user": "ppo_hd_plan.pdf"
    }
    metadata_filter= {
        "equals": {
            "key": "file_name",
            "value": filter_mappings[filter_type]
        }
    }
    return metadata_filter
    


In [7]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

In [8]:
def get_retrieval_config(numResults, metadata_filter):
    retrieval_config = {
        'vectorSearchConfiguration': {
            'numberOfResults': numResults,
             'overrideSearchType': "HYBRID",
            'filter': metadata_filter
        }
    }
    return retrieval_config

In [9]:
def retrieveAndGenerate(input, kbId, metadata_filter, numResults=5, sessionId=None, model_id = "anthropic.claude-instant-v1", region_id = "us-east-1"):
    model_arn = f'arn:aws:bedrock:{region_id}::foundation-model/{model_id}'
    if sessionId:
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kbId,
                    'modelArn': model_arn,
                    'retrievalConfiguration':get_retrieval_config(numResults,metadata_filter)
                }
            },
            sessionId=sessionId
        )
    else:
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kbId,
                    'modelArn': model_arn,
                    'retrievalConfiguration':get_retrieval_config(numResults,metadata_filter)
                }
            }
        )

In [20]:
#retreiveandgenerate
query = "What is the deductible for a family for in-network?"
metadata_filter = create_filter("ppo_hd_user") 

In [21]:
response = retrieveAndGenerate(query, plan_kb_id,metadata_filter, model_id=modelId)
generated_text = response['output']['text']
pp.pprint(generated_text)

'The in-network family deductible is $7,500 per plan year.'
